In [4]:
!pip install opencv-python numpy matplotlib

Defaulting to user installation because normal site-packages is not writeable
  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl (39.0 MB)



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# CREACIÓN DEL FONDO ESTÁTICO
import cv2
import numpy as np
import matplotlib.pyplot as plt

video = cv2.VideoCapture("trafico.mp4")
# Variables para acumular frames
accumulated_frame = None
frame_count = 0

while True:
    ret, frame = video.read()
    if not ret:
        break  # Fin del video

    cv2.imshow('Video', frame)  # Muestra el frame en una ventana

    # Espera 25 ms y sale si se presiona la tecla 'q'
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

    # Convierte el frame a float32 para acumulación
    frame_float = frame.astype(np.float32)

    if accumulated_frame is None:
        accumulated_frame = frame_float
    else:
        accumulated_frame += frame_float
    frame_count += 1

video.release()

# Calcula el promedio
background = accumulated_frame / frame_count

# Convierte de nuevo a uint8 para mostrar/guardar
background_uint8 = cv2.convertScaleAbs(background)

# Muestra el fondo promedio
cv2.imshow('Fondo Promedio', background_uint8)
# cv2.imwrite('fondo_estatico.jpg', background_uint8)
cv2.waitKey(0)
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'cv2'

In [8]:
# UMBRALIZACIÓN Y BINARIZACIÓN DEL VIDEO
import cv2
import numpy as np
import matplotlib.pyplot as plt

fondo = cv2.imread("fondo_estatico.jpg") #Leer la imagen del fondo.
fondo_BW = cv2.cvtColor(fondo, cv2.COLOR_BGR2GRAY) #Pasar el fondo a blanco y negro.

video = cv2.VideoCapture("trafico.mp4")
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))

# -------------------- VARIABLES PARA EL CONTADOR --------------------
contador = 0
linea_conteo = frame_height // 2   # Línea horizontal de conteo (invisible)
offset = 10                        # Margen de tolerancia al cruzar la línea
bloqueados = []                    # Lista de puntos bloqueados temporalmente
tiempo_bloqueo = 15                # Cuántos frames permanece bloqueado un punto

# Definir línea diagonal de conteo
p1a = (100, frame_height - 100)      # punto inicial (abajo izquierda)
p1b = (frame_width - 100, 100)       # punto final (arriba derecha)
# Definir línea 2 (fin)
p2a = (200, frame_height - 50)
p2b = (frame_width - 100, 200)

# Calcular coeficientes de ambas líneas (Ax + By + C = 0)
def coef_linea(p1, p2):
    A = p1[1] - p2[1]
    B = p2[0] - p1[0]
    C = p1[0]*p2[1] - p2[0]*p1[1]
    return A, B, C

A1, B1, C1 = coef_linea(p1a, p1b)
A2, B2, C2 = coef_linea(p2a, p2b)

# Diccionario para guardar tiempos de entrada de cada vehículo
vehiculos_tiempo = {}
vehiculo_id = 0
# --------------------------------------------------------------------

while True:
    ret, frame = video.read()
    if not ret:
        break

    # Convertir el frame a escala de grises
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Restar el fondo al frame actual (diferencia absoluta)
    diff = cv2.absdiff(frame_gray, fondo_BW)

    # Aplicar un umbral para detectar movimiento (valores mayores a 30, por ejemplo)
    T = 30
    _, thresh = cv2.threshold(diff, T, 255, cv2.THRESH_BINARY)
    #cv2.imshow('Diferencia Absoluta', diff)
    #cv2.imshow('Movimiento Detectado', thresh)

# DETECCIÓN DE BLOBS EN EL VIDEO    

    # Limpiar ruido con operaciones morfológicas
    kernel = np.ones((5, 5), np.uint8)
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_DILATE, kernel)
    contours, _ = cv2.findContours(cleaned, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    nuevos_centros = []  # Centros detectados en este frame

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 300:
            continue  # Ignoramos áreas pequeñas

        x, y, w, h = cv2.boundingRect(cnt)

        # Adaptar área mínima en función de la posición vertical (y)
        y_center = y + h // 2
        vertical_factor = y_center / frame_height
        min_area = 300 + (vertical_factor) * 500  # dinámico: 300 a 1800

        if area > min_area:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Calcular el centro del contorno
            cx = x + w // 2
            cy = y + h // 2
            nuevos_centros.append((cx, cy))
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)

    # -------------------- CONTEO DE VEHÍCULOS --------------------
    # Actualizar "bloqueados": disminuir contador de frames de vida
    bloqueados = [(bx, by, t - 1) for (bx, by, t) in bloqueados if t > 1]

    for (cx, cy) in nuevos_centros:
        if (cy > linea_conteo - offset) and (cy < linea_conteo + offset):
            # Verificar si está cerca de un punto bloqueado (ya contado)
            if not any(abs(cx - bx) < 50 and abs(cy - by) < 50 for (bx, by, _) in bloqueados):
                contador += 1
                bloqueados.append((cx, cy, tiempo_bloqueo))
    
    # Dibujar línea diagonal de conteo
    cv2.line(frame, p1a, p1b, (255, 0, 0), 2)

    # Mostrar el contador en pantalla
    cv2.putText(frame, f'Contador: {contador}', (20, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
    # ----------------------------------------------------------------------

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

#cv2.imshow('Frame con Blobs', frame)
#cv2.imshow('Máscara de Movimiento', cleaned)
# RESULTADO
    cv2.imshow('Deteccion de Coches', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

In [ ]:
#hola
#adios